In [1]:
#parsing the whole of CAZy
import re

class CAZy_data:
    def __init__(self,filename1,filename2):
        self.data,self.acc,self.seq=[],[],[]
        with open(filename1,'r',encoding='utf-8') as inpt:
            for each in inpt:
                self.data.append(each.rstrip().split('$'))
                
        with open(filename2,'r',encoding='utf-8') as inpt1:
            for each1 in inpt1:
                if each1.startswith('>'):
                    self.acc.append(each1.rstrip())
                else:
                    self.seq.append(each1.rstrip())
            
                  
    def data_fetch(self,typ,position):
        typ_data=[]
        if typ=='all':
            typ_data=self.data
        else:
            for each in self.data:
                mult=each[position].split(' ')
                if len(mult)==1:#### In case typ = EC, Multi EC number and protein with no EC number are ignore.
                    if mult[0]==typ:
                        typ_data.append(each)
        return typ_data
    
    def EC_GH(self,ec_no,gh_fam):
        self.fasta=[]
        cazy_ec=self.data_fetch(ec_no,1)
        cazy_gh=self.data_fetch(gh_fam,-1)
        self.common_data=[[i[0],i[1],i[3],i[4],i[-2],i[-1]] for i in cazy_ec if i in cazy_gh]
        rm_prt, rm_prt_fasta=[],[]
        for each in range(len(self.common_data)):
            t=self.common_data[each]
            if self.prtn_filter(t[0]):
                all_acc=t[3].split(' ')
                if all_acc[0]!='':
                    for e_acc in all_acc:
                        e_seq=self.seq_fetch(e_acc)
                        try:
                            create_error=0/len(e_seq) # to remove accession number which doesnt have hits
                            self.fasta.append(f'>{e_acc}${t[0]}${t[1]}${t[2]}${t[-2]}${t[-1]}')
                            self.fasta.append(e_seq[0])
                        except ZeroDivisionError:
                            rm_prt_fasta.append(e_acc)
            else:
                rm_prt.append(t)
#         print('Total number of sequences:',len(self.fasta)/2)
#         print('Number of removed partial or fragment proteins (CAZy):',len(rm_prt))
#         print('Number of removed partial or fragment proteins (Fasta):',len(rm_prt_fasta))
        return self.fasta,rm_prt,rm_prt_fasta
    def prtn_filter(self,prt_name):
        hit=1
        if re.search('partial|fragment',prt_name.lower()):
            hit-=1
        return hit
            
    def seq_fetch(self,accession):
        hits=[]
        temp=0
        for each in range(len(self.acc)):
            if re.search(f'{accession}\D',self.acc[each]):
                temp+=1
                
                if self.prtn_filter(self.acc[each]): # remove partial| fragment accession numbers from GenBank description
                    hits.append(self.seq[each])
        if temp>1:
            print(f'Multiple hits for {accession}')
        elif temp==0:
            print(f'No hits for {accession}')
        return hits
     
In_data=CAZy_data('cazy_char_10_6_22.txt','char_gh_23_6_22.txt')


In [3]:
EC_number= input('please input the EC number (write all if no preference):')
GH_family= input('please input the GH family (write all if no preference):')


please input the EC number (write all if no preference):3.2.1.4
please input the GH family (write all if no preference):GH8


In [4]:
cazy_acc_seq,rm_cazy,rm_genbank=In_data.EC_GH(EC_number,GH_family) # write all to fetch all the EC number or all the GH family

No hits for NP_213966.1


In [17]:
import pandas as pd
import os
import numpy as np

In [5]:
#splitting sequence and accession IDs
acc,seq=[],[]
for ii in cazy_acc_seq:
    if ii.startswith('>'):
        acc.append(ii.rstrip())
    else:
        seq.append(ii.rstrip()) 

In [6]:
#creating a dictionary with only the reqiuired information with the accession ID
acc_seq={}
for i in range(len(acc)):
    accc=acc[i].split('$')
    acc_no=accc[0]
    x='>'+acc_no[1:]+'&'+accc[2]+'&'+accc[4]+'&'+accc[5]
    acc_seq[x] = seq[i]


In [7]:
#the file with the required info
filename='EC'+EC_number.split('.')[3]+'_'+GH_family+'.fasta'
y=len(seq)
with open('EC4_GH8.fasta','w',encoding='utf-8') as otp:
    for i in acc_seq.keys():
        seq=acc_seq.get(i)
        otp.write(i+'\n'+seq+'\n')
    

In [ ]:
#perform CD-hit step
cmd='cd-hit -i EC4_GH8.fasta -o EC4_GH8_50.fasta -c 0.50 -n 3'
os.system(cmd)

In [8]:
acc,seq=[],[]
with open('EC4_GH8_50.fasta','r',encoding='utf-8') as fasta_file:
    for ii in fasta_file:
        if ii.startswith('>'):
            acc.append(ii.rstrip())
            
        else:
            seq.append(ii.rstrip()) 

In [11]:
acc_seq={}
for i in range(len(acc)):
    accc=acc[i].split('$')
    acc_no=accc[0]
    acc_seq[acc_no[1:]] = seq[i]
acc_seq

{'ABN51508.1&3.2.1.4&Bacteria&GH8': 'MKNVKKRVGVVLLILAVLGVYMLAMPANTVSAAGVPFNTKYPYGPTSIADNQSEVTAMLKAEWEDWKSKRITSNGAGGYKRVQRDASTNYDTVSEGMGYGLLLAVCFNEQALFDDLYRYVKSHFNGNGLMHWHIDANNNVTSHDGGDGAATDADEDIALALIFADKLWGSSGAINYGQEARTLINNLYNHCVEHGSYVLKPGDRWGGSSVTNPSYFAPAWYKVYAQYTGDTRWNQVADKCYQIVEEVKKYNNGTGLVPDWCTASGTPASGQSYDYKYDATRYGWRTAVDYSWFGDQRAKANCDMLTKFFARDGAKGIVDGYTIQGSKISNNHNASFIGPVAAASMTGYDLNFAKELYRETVAVKDSEYYGYYGNSLRLLTLLYITGNFPNPLSDLSGQPTPPSNPTPSLPPQVVYGDVNGDGNVNSTDLTMLKRYLLKSVTNINREAADVNRDGAINSSDMTILKRYLIKSIPHLPY',
 'AAA22409.1&3.2.1.4&Bacteria&GH8': 'MVEKRKIFTVLCACGIGFTSYTSCISAAAIDNDTLINNGHKINSSIITNSSQVSAVAKEMKPFPQQVNYSGILKPNHVSQESLNNAVKNYYNDWKKKYLKNDLSSLPGGYYVKGEITGNPDGFRPLGTSEGQGYGMIITVLMAGHDSNAQTIYDGLFKTARAFKSSINPNLMGWVVADDKKAQGHFDSATDGDLDIAYSLLLAHKQWGSSGKINYLKEAQNMITKGIKASNVTKNNGLNLGDWGDKSTFDTRPSDWMMSHLRAFYEFTGDKTWLNVIDNLYNTYTNFTNKYSPKTGLISDFVVKNPPQPAPKDFLDESKYTDSYYYNASRVPLRIVMDYAMYGEKRGKVISDKVATWIKSKTKGNPSKIVDGYKLDGTNIGDYPTAVYVSPFIAAGTTNSKNQEWVNSGWDWMKNKKESYFSDSYNLLTMLFLTGNWWKPIPDEKK

In [12]:
acc_vs_seq=pd.DataFrame.from_dict(acc_seq, orient = 'index')
acc_vs_seq.set_axis(['Sequence'], axis=1 , inplace=True)
acc_vs_seq

,Sequence
ABN51508.1&3.2.1.4&Bacteria&GH8,MKNVKKRVGVVLLILAVLGVYMLAMPANTVSAAGVPFNTKYPYGPT...
AAA22409.1&3.2.1.4&Bacteria&GH8,MVEKRKIFTVLCACGIGFTSYTSCISAAAIDNDTLINNGHKINSSI...
CAB89803.1&3.2.1.4&Bacteria&GH8,MMAMFRAGIILLALVLGLSQASAAEGWGTFKTRFLTSEGRIQDTGN...
ACX73853.1&3.2.1.4&Bacteria&GH8,MLQRYPTRDLFVEAGYGPNFADQLIQNAYSKLFEGDPIDERIYFEA...
ABU45499.1&3.2.1.4&Bacteria&GH8,MIFDMLSNYLLGWMGEIFFLKGIHMKNLAKVMFGVAAVAAVTASAG...
ACN29537.1&3.2.1.4&Bacteria&GH8,MNGKSVKAAVIYLLGGLLVLLGLASMPASANANFAVGDPGWEAYKK...
AAA16969.1&3.2.1.4&Bacteria&GH8,MSVMAAMGGAQVLSSTGAFADPAPDAVAQQWAIFRAKYLRPSGRVV...
BAB69035.1&3.2.1.4&Bacteria&GH8,MKVFRNSIIRKSAVLFCAVLLILPAGLSLAANKPFPQHTSYTSGSI...
AAO43176.1&3.2.1.4&Bacteria&GH8,MPRVLHYLIPTLLWLWASQATAAVCDWPAWEQYKQHYISAEGRVID...
CAD90973.1&3.2.1.4&Bacteria&GH8,MRRWRALLLAASVAVAPGLPATAQQAMINADAWSAYKAKFLDPSGR...


In [13]:
import numpy as np
acc_vs_seq['Length']=np.nan
acc_vs_seq

,Sequence,Length
ABN51508.1&3.2.1.4&Bacteria&GH8,MKNVKKRVGVVLLILAVLGVYMLAMPANTVSAAGVPFNTKYPYGPT...,NaN
AAA22409.1&3.2.1.4&Bacteria&GH8,MVEKRKIFTVLCACGIGFTSYTSCISAAAIDNDTLINNGHKINSSI...,NaN
CAB89803.1&3.2.1.4&Bacteria&GH8,MMAMFRAGIILLALVLGLSQASAAEGWGTFKTRFLTSEGRIQDTGN...,NaN
ACX73853.1&3.2.1.4&Bacteria&GH8,MLQRYPTRDLFVEAGYGPNFADQLIQNAYSKLFEGDPIDERIYFEA...,NaN
ABU45499.1&3.2.1.4&Bacteria&GH8,MIFDMLSNYLLGWMGEIFFLKGIHMKNLAKVMFGVAAVAAVTASAG...,NaN
ACN29537.1&3.2.1.4&Bacteria&GH8,MNGKSVKAAVIYLLGGLLVLLGLASMPASANANFAVGDPGWEAYKK...,NaN
AAA16969.1&3.2.1.4&Bacteria&GH8,MSVMAAMGGAQVLSSTGAFADPAPDAVAQQWAIFRAKYLRPSGRVV...,NaN
BAB69035.1&3.2.1.4&Bacteria&GH8,MKVFRNSIIRKSAVLFCAVLLILPAGLSLAANKPFPQHTSYTSGSI...,NaN
AAO43176.1&3.2.1.4&Bacteria&GH8,MPRVLHYLIPTLLWLWASQATAAVCDWPAWEQYKQHYISAEGRVID...,NaN
CAD90973.1&3.2.1.4&Bacteria&GH8,MRRWRALLLAASVAVAPGLPATAQQAMINADAWSAYKAKFLDPSGR...,NaN


In [14]:
for i in range(len(acc)): 
    acc_vs_seq.iloc[i,1]=int(len(acc_vs_seq.iloc[i,0]))


In [15]:
filename='EC4_GH8_50.csv'
acc_vs_seq.to_csv(filename)

In [16]:
#finding out the outliers and removing them step follows

In [20]:
acc_vs_seq=pd.read_csv('EC4_GH8_50.csv') #read the csv file containing the length of each sequence as well, obtained from Get_EC_GH.ipynb
acc_vs_seq.rename( columns={'Unnamed: 0':'ID'}, inplace=True ) #rename the unnamed column as ID
acc_vs_seq

,ID,Sequence,Length
0,ABN51508.1&3.2.1.4&Bacteria&GH8,MKNVKKRVGVVLLILAVLGVYMLAMPANTVSAAGVPFNTKYPYGPT...,477.0
1,AAA22409.1&3.2.1.4&Bacteria&GH8,MVEKRKIFTVLCACGIGFTSYTSCISAAAIDNDTLINNGHKINSSI...,463.0
2,CAB89803.1&3.2.1.4&Bacteria&GH8,MMAMFRAGIILLALVLGLSQASAAEGWGTFKTRFLTSEGRIQDTGN...,333.0
3,ACX73853.1&3.2.1.4&Bacteria&GH8,MLQRYPTRDLFVEAGYGPNFADQLIQNAYSKLFEGDPIDERIYFEA...,371.0
4,ABU45499.1&3.2.1.4&Bacteria&GH8,MIFDMLSNYLLGWMGEIFFLKGIHMKNLAKVMFGVAAVAAVTASAG...,752.0
5,ACN29537.1&3.2.1.4&Bacteria&GH8,MNGKSVKAAVIYLLGGLLVLLGLASMPASANANFAVGDPGWEAYKK...,350.0
6,AAA16969.1&3.2.1.4&Bacteria&GH8,MSVMAAMGGAQVLSSTGAFADPAPDAVAQQWAIFRAKYLRPSGRVV...,342.0
7,BAB69035.1&3.2.1.4&Bacteria&GH8,MKVFRNSIIRKSAVLFCAVLLILPAGLSLAANKPFPQHTSYTSGSI...,407.0
8,AAO43176.1&3.2.1.4&Bacteria&GH8,MPRVLHYLIPTLLWLWASQATAAVCDWPAWEQYKQHYISAEGRVID...,371.0
9,CAD90973.1&3.2.1.4&Bacteria&GH8,MRRWRALLLAASVAVAPGLPATAQQAMINADAWSAYKAKFLDPSGR...,347.0


In [21]:
acc_vs_seq.describe() #statistical description of the dataset


,Length
count,12.000000
mean,418.833333
std,118.181550
min,333.000000
25%,345.750000
50%,371.000000
75%,465.750000
max,752.000000


In [22]:
#create a function to find outliers using IQR

def find_outliers_IQR(df):
    q1=df.quantile(0.25) #finding the first and 3rd quatrile
    q3=df.quantile(0.75)
    IQR=q3-q1 #finding the interquatrile range
    outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))] #finding outliers
    return outliers
x=list(find_outliers_IQR(acc_vs_seq['Length'])) #finding length outliers, saving them to list
x

[752.0]

In [23]:
acc_vs_seq=acc_vs_seq[acc_vs_seq.Length.isin(x) == False] #removing the outlier. if the length is not in hte outlier list (x), they are taken
acc_vs_seq.reset_index(inplace = True, drop = True) #resetting the index, as after dropping there are missing index values
acc_vs_seq

,ID,Sequence,Length
0,ABN51508.1&3.2.1.4&Bacteria&GH8,MKNVKKRVGVVLLILAVLGVYMLAMPANTVSAAGVPFNTKYPYGPT...,477.0
1,AAA22409.1&3.2.1.4&Bacteria&GH8,MVEKRKIFTVLCACGIGFTSYTSCISAAAIDNDTLINNGHKINSSI...,463.0
2,CAB89803.1&3.2.1.4&Bacteria&GH8,MMAMFRAGIILLALVLGLSQASAAEGWGTFKTRFLTSEGRIQDTGN...,333.0
3,ACX73853.1&3.2.1.4&Bacteria&GH8,MLQRYPTRDLFVEAGYGPNFADQLIQNAYSKLFEGDPIDERIYFEA...,371.0
4,ACN29537.1&3.2.1.4&Bacteria&GH8,MNGKSVKAAVIYLLGGLLVLLGLASMPASANANFAVGDPGWEAYKK...,350.0
5,AAA16969.1&3.2.1.4&Bacteria&GH8,MSVMAAMGGAQVLSSTGAFADPAPDAVAQQWAIFRAKYLRPSGRVV...,342.0
6,BAB69035.1&3.2.1.4&Bacteria&GH8,MKVFRNSIIRKSAVLFCAVLLILPAGLSLAANKPFPQHTSYTSGSI...,407.0
7,AAO43176.1&3.2.1.4&Bacteria&GH8,MPRVLHYLIPTLLWLWASQATAAVCDWPAWEQYKQHYISAEGRVID...,371.0
8,CAD90973.1&3.2.1.4&Bacteria&GH8,MRRWRALLLAASVAVAPGLPATAQQAMINADAWSAYKAKFLDPSGR...,347.0
9,CBL17008.1&3.2.1.4&Bacteria&GH8,MKLRRKTAVVMAAGAVMMANAAPVLTAAEVKEMQTGALSEETVQFY...,474.0


In [24]:
filename='EC4_GH8_50_No_outliers.csv' #saving the updated csv file
acc_vs_seq.to_csv(filename)
print('csv file with no outliers is successfully saved')

csv file with no outliers is successfully saved


In [26]:
#saving the sequences to multifasta
xx=(acc_vs_seq.ID.count())-1
with open('EC4_GH8_50_no_outlier.fasta','w',encoding='utf-8') as otp:
    for i in range(xx):
        acc_id=acc_vs_seq.iloc[i,0]
        seq=acc_vs_seq.iloc[i,1]
        otp.write('>'+acc_id+'\n'+seq+'\n')
print('multifasta file with no length outliers is successfully saved')

multifasta file with no length outliers is successfully saved
